In [12]:
import gzip
import json
import pandas as pd
from bs4 import BeautifulSoup
import pickle


#### 加载纯纯的原始数据

In [13]:
f = open('database.json.gz', mode='rb')
database = json.loads(gzip.decompress(f.read()))
f.close()
del f
df = pd.DataFrame()
for k, v in database.items():
    temp = pd.DataFrame(database[k])
    temp['nfxq'] = k
    del temp['xh']
    df = pd.concat([df, temp])
df.reset_index(inplace=True)
del df['index']
df.head(3)
# 序号、课程号、课程名称、课程体系名、开课系所名称、教学班号、学分、执行计划编号、起止周、上课时间、教师、备注


,kch,kcmc,kctxm,kkxsmc,jxbh,xf,zxjhbh,qzz,sksj,teacher,bz,nfxq
0,00431641,量子力学讨论班,专业必修,物理学院,1,2,BZ1314100431641_15296,1-16,<p>星期一(第3节-第4节)</p>,<p>马中水</p><p>刘玉鑫</p><p>马伯强</p><p>孟杰</p><p>陈斌</...,--补录,13-14-1
1,00432150,量子力学 (A),专业必修,物理学院,1,4,BZ1314100432150_18727,1-16,<p>星期一(第1节-第2节)</p>,<p>李重生</p><p>田光善</p><p>刘玉鑫</p><p>朱世琳</p>,--补录,13-14-1
2,04130002,体育二,体育,体育教研部,1,1,BZ1314104130002_11015,1-16,<p>星期一(第1节-第2节)</p>,<p>萧文革</p>,--补录,13-14-1


#### 上面是纯纯的原始数据，下面开始清洗

#### 洗掉html标记

In [14]:
df['sksj'] = df['sksj'].apply(lambda x: BeautifulSoup(
    x, 'html.parser').get_text(',').split(','))
df['teacher'] = df['teacher'].apply(lambda x: BeautifulSoup(
    x, 'html.parser').get_text(',').split(','))


In [ ]:
df.head(3)


,kch,kcmc,kctxm,kkxsmc,jxbh,xf,zxjhbh,qzz,sksj,teacher,bz,nfxq
0,00431641,量子力学讨论班,专业必修,物理学院,1,2,BZ1314100431641_15296,1-16,[星期一(第3节-第4节)],"[马中水, 刘玉鑫, 马伯强, 孟杰, 陈斌, 朱守华, 朱世琳, 曹庆宏, 杨李林, 全海...",--补录,13-14-1
1,00432150,量子力学 (A),专业必修,物理学院,1,4,BZ1314100432150_18727,1-16,[星期一(第1节-第2节)],"[李重生, 田光善, 刘玉鑫, 朱世琳]",--补录,13-14-1
2,04130002,体育二,体育,体育教研部,1,1,BZ1314104130002_11015,1-16,[星期一(第1节-第2节)],[萧文革],--补录,13-14-1


#### 年份学期洗成两半。年份只记录第一年。由于全部都可转化成int，就都转成int。

In [ ]:
df['nfxq'].unique()


array(['13-14-1', '13-14-2', '13-14-3', '14-15-1', '14-15-2', '14-15-3',
       '15-16-1', '15-16-2', '15-16-3', '16-17-1', '16-17-2', '16-17-3',
       '17-18-1', '17-18-2', '17-18-3', '18-19-1', '18-19-2', '18-19-3',
       '19-20-1', '19-20-2', '19-20-3', '20-21-1', '20-21-2', '20-21-3',
       '21-22-1', '21-22-2', '21-22-3', '22-23-1', '22-23-2', '22-23-3',
       '12-13-2', '12-13-3', '23-24-1'], dtype=object)

In [ ]:
temp = df['nfxq'].str.split('-')
xq = temp.apply(lambda x: int(x[2]))
qishinian = temp.apply(lambda x: int(x[0]))  # 起始年
df['xq'] = xq
df['qsn'] = qishinian
del df['nfxq']
df.head(3)


,kch,kcmc,kctxm,kkxsmc,jxbh,xf,zxjhbh,qzz,sksj,teacher,bz,xq,qsn
0,00431641,量子力学讨论班,专业必修,物理学院,1,2,BZ1314100431641_15296,1-16,[星期一(第3节-第4节)],"[马中水, 刘玉鑫, 马伯强, 孟杰, 陈斌, 朱守华, 朱世琳, 曹庆宏, 杨李林, 全海...",--补录,1,13
1,00432150,量子力学 (A),专业必修,物理学院,1,4,BZ1314100432150_18727,1-16,[星期一(第1节-第2节)],"[李重生, 田光善, 刘玉鑫, 朱世琳]",--补录,1,13
2,04130002,体育二,体育,体育教研部,1,1,BZ1314104130002_11015,1-16,[星期一(第1节-第2节)],[萧文革],--补录,1,13


#### 起止周洗成两半。注意split后的空字符串不可转成int，因此统一转成0。

In [ ]:
df['qzz'].unique()


array(['1-16', '9-16', '2-16', '1-10', '1-15', '1-12', '10-15', '10-14',
       '9-14', '1-9', '1-8', '1-11', '1-14', '1-7', '5-16', '11-16',
       '7-14', '3-16', '8-16', '6-16', '5-10', '12-16', '10-16', '1-6',
       '2-11', '13-16', '7-16', '2-10', '4-16', '7-15', '3-15', '14-16',
       '1-3', '4-12', '7-8', '1-4', '2-3', '3-4', '1-2', '2-4', '1-1',
       '3-3', '1-5', '2-2', '2-5', '3-8', '8-19', '1-13', '2-12', '6-12',
       '5-7', '2-8', '3-5', '7-17', '4-9', '3-10', '8-13', '-', '2-15',
       '0-3', '4-4', '0-1', '4-5', '5-5', '0-2', '11-15', '6-14', '6-6',
       '8-14', '6-13', '0-', '15-16', '1-17', '7-12', '5-6', '0-0', '3-7',
       '0-4', '9-12', '5-12', '3-14', '8-9', '3-12', '2-14', '1-18',
       '10-18', '5-18', '9-15', '5-8', '2-13', '8-15', '4-11', '4-13',
       '9-18', '4-15', '3-13', '7-10', '4-17', '2-7', '15-15', '14-14',
       '11-14', '8-8', '9-9', '10-10', '2-9', '13-13', '11-11', '10-17',
       '3-11', '12-12', '12-15', '5-15', '6-15', '16-16', '9-10

In [ ]:
def safe_trans_int(s:str)->int:
    try:
        return(int(s))
    except ValueError:
        return 0

temp = df['qzz'].str.split('-')
df['qsz'] = temp.apply(lambda x: safe_trans_int(x[0]))
df['zzz'] = temp.apply(lambda x: safe_trans_int(x[1]))
del df['qzz']
df.head(3)


,kch,kcmc,kctxm,kkxsmc,jxbh,xf,zxjhbh,sksj,teacher,bz,xq,qsn,qsz,zzz
0,00431641,量子力学讨论班,专业必修,物理学院,1,2,BZ1314100431641_15296,[星期一(第3节-第4节)],"[马中水, 刘玉鑫, 马伯强, 孟杰, 陈斌, 朱守华, 朱世琳, 曹庆宏, 杨李林, 全海...",--补录,1,13,1,16
1,00432150,量子力学 (A),专业必修,物理学院,1,4,BZ1314100432150_18727,[星期一(第1节-第2节)],"[李重生, 田光善, 刘玉鑫, 朱世琳]",--补录,1,13,1,16
2,04130002,体育二,体育,体育教研部,1,1,BZ1314104130002_11015,[星期一(第1节-第2节)],[萧文革],--补录,1,13,1,16


#### 最后展示一下就dump吧！

In [ ]:
df.head(10)
# 课程号、课程名称、课程体系名、开课系所名称、教学班号、学分、执行计划编号、上课时间、教师、备注、学期、起始年、起始周、终止周


,kch,kcmc,kctxm,kkxsmc,jxbh,xf,zxjhbh,sksj,teacher,bz,xq,qsn,qsz,zzz
0,00431641,量子力学讨论班,专业必修,物理学院,1,2,BZ1314100431641_15296,[星期一(第3节-第4节)],"[马中水, 刘玉鑫, 马伯强, 孟杰, 陈斌, 朱守华, 朱世琳, 曹庆宏, 杨李林, 全海...",--补录,1,13,1,16
1,00432150,量子力学 (A),专业必修,物理学院,1,4,BZ1314100432150_18727,[星期一(第1节-第2节)],"[李重生, 田光善, 刘玉鑫, 朱世琳]",--补录,1,13,1,16
2,04130002,体育二,体育,体育教研部,1,1,BZ1314104130002_11015,[星期一(第1节-第2节)],[萧文革],--补录,1,13,1,16
3,01339180,世界文化地理,通选课,城市与环境学院,1,2,zxjhbk201300053494,[星期三(第10节-第11节)],[邓辉],,1,13,1,16
4,04130620,定向与徒步运动,体育,体育教研部,1,1,zxjhbk201300056667,[星期一(第5节-第6节)],[钱俊伟],一体,1,13,1,16
5,04130030,太极拳,体育,体育教研部,1,1,zxjhbk201300056929,[星期一(第1节-第2节)],[刘林青],男生班 五四跑廊 刘林青老师授课,1,13,1,16
6,04130090,篮球,体育,体育教研部,1,1,zxjhbk201300056936,[星期一(第7节-第8节)],[张剑],第二体育馆东篮球场，男生班,1,13,1,16
7,01832360,传播伦理学,专业任选,新闻与传播学院,1,2,zxjhbk201300057729,[星期三(第5节-第6节)],[陈汝东],,1,13,1,16
8,04332120,影视音乐,专业必修,艺术学院,1,2,zxjhbk201300060006,[星期二(第10节-第11节)],[李静],,1,13,1,16
9,04332551,艺术训练（一）,全校公选课,艺术学院,1,1,zxjhbk201300060010,[],[],限艺术特长生选课,1,13,1,16


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60569 entries, 0 to 60568
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   kch      60569 non-null  object
 1   kcmc     60569 non-null  object
 2   kctxm    60569 non-null  object
 3   kkxsmc   60569 non-null  object
 4   jxbh     60569 non-null  object
 5   xf       60569 non-null  object
 6   zxjhbh   60569 non-null  object
 7   sksj     60569 non-null  object
 8   teacher  60569 non-null  object
 9   bz       60569 non-null  object
 10  xq       60569 non-null  int64 
 11  qsn      60569 non-null  int64 
 12  qsz      60569 non-null  int64 
 13  zzz      60569 non-null  int64 
dtypes: int64(4), object(10)
memory usage: 6.5+ MB


In [ ]:
with open('df.pkl', 'wb') as f:
    pickle.dump(df, f)
